In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('marketing_customer_analysis.csv')

# Show the dataframe shape

In [6]:
df.shape

(10910, 26)

# Standardize header names

In [12]:
cols = []
for i in list(df.columns):
    cols.append(i.lower().replace(' ', '_').replace(':', ''))
df.columns = cols
df.columns

Index(['unnamed_0', 'customer', 'state', 'customer_lifetime_value', 'response',
       'coverage', 'education', 'effective_to_date', 'employmentstatus',
       'gender', 'income', 'location_code', 'marital_status',
       'monthly_premium_auto', 'months_since_last_claim',
       'months_since_policy_inception', 'number_of_open_complaints',
       'number_of_policies', 'policy_type', 'policy', 'renew_offer_type',
       'sales_channel', 'total_claim_amount', 'vehicle_class', 'vehicle_size',
       'vehicle_type'],
      dtype='object')

# Which columns are numerical?

In [18]:
num_cols = list(df.select_dtypes([np.number]).columns)
print(f'The columns {num_cols} are numerical.')

The columns ['unnamed_0', 'customer_lifetime_value', 'income', 'monthly_premium_auto', 'months_since_last_claim', 'months_since_policy_inception', 'number_of_open_complaints', 'number_of_policies', 'total_claim_amount'] are numerical.


# Which columns are categorical?

In [22]:
cat_cols = list(df.select_dtypes(exclude=['number','bool_']).columns)
print(f'The columns {cat_cols} are categorical.')

The columns ['customer', 'state', 'response', 'coverage', 'education', 'effective_to_date', 'employmentstatus', 'gender', 'location_code', 'marital_status', 'policy_type', 'policy', 'renew_offer_type', 'sales_channel', 'vehicle_class', 'vehicle_size', 'vehicle_type'] are categorical.


# Check and deal with NaN values

First, we will remove all rows in which all values are NaN.

In [27]:
df = df.dropna(how='all')
df.reset_index(drop=True, inplace=True)

Then, we'll deal with NaN values in numerical columns. We'll be replacing them by the median of their own column, because, although the mean and median are not very different, the former might be influed by outliers.

In [28]:
med_months = df['months_since_last_claim'].median()
med_num = df['number_of_open_complaints'].median()
df['months_since_last_claim'] = df['months_since_last_claim'].fillna(med_months)
df['number_of_open_complaints'] = df['number_of_open_complaints'].fillna(med_num)

Finally, we'll adress the NaN values in categorical columns. In this case, we'll replace them by the string 'Unknown'. Since these are the only NULLS pending, we can just apply the function fillna() to the whole DataFrame.

In [29]:
df = df.fillna('Unknown')

In [30]:
#snapshot table for handling nulls
nulls_df = pd.DataFrame(round(df.isna().sum()/len(df), 4) * 100)
nulls_df = nulls_df.rename(columns={'index':'header', 0: 'prop_nulls'})
nulls_df.index.name = 'header'
nulls_df

,prop_nulls
header,
unnamed_0,0.0
customer,0.0
state,0.0
customer_lifetime_value,0.0
response,0.0
coverage,0.0
education,0.0
effective_to_date,0.0
employmentstatus,0.0


# Datetime format - Extract the months from the dataset and store in a separate column

First, we will import the necessary modules and convert the column effective_to_date to datetime type.

In [33]:
from datetime import date

In [36]:
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'], errors='coerce')

Then, we will create a new column in our dataframe with the value of the month for each row, which we will have extracted from the effective_to_date column.

In [37]:
df['month'] = df['effective_to_date'].dt.strftime('%m')

## Filter the data to show only the information for the first quarter

Once we have extracted the month for each row and stored it in a new column, we will filter the dataframe and create a new one with only the results from the frist quarter (months 01, 02 and 03).

In [54]:
months = ['01', '02', '03']
first_quarter = df[df.month.isin(months)]
first_quarter.head()

,unnamed_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,Unknown,02
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,Unknown,01
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,02
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,01
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,Unknown,01


# BONUS: Put all the previously mentioned data transformations into a function